## An Overview of the Machine Learning Life Cycle

- https://github.com/PacktPublishing/Feature-Store-for-Machine-Learning/blob/main/Chapter01/data_exploration.ipynb

### The ML life cycle in practice

Problem statement
- customer segments and customer lifetime value (LTV)

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("data/OnlineRetail.csv", encoding="unicode_escape")

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


• InvoiceNo: 고유 주문 ID(데이터는 정수 유형)

• StockCode: 제품의 고유 ID, 데이터는 문자열 유형입니다.

• Description: 제품의 고유 ID, 데이터는 문자열 유형입니다.

• Quantity: 주문한 제품의 단위 수입니다.

• InvoiceDate: The date when the invoice was generated

• UnitPrice: 인보이스가 생성된 날짜입니다.

• CustomerID: 제품을 주문한 고객의 고유 ID입니다.

• Country: The country where the product was ordered

이 연습에서는 월별 수익을 계산하고 계절성을 살펴보겠습니다. 

다음 코드 블록은 InvoiceDate 열에서 연도 및 월(yyyymm) 정보를 추출하고, 단위 가격 및 수량 열을 곱하여 각 거래의 수익 속성을 계산한 다음, 추출된 연도-월(yyyymm) 열을 기준으로 수익을 집계합니다.앞의 코드 문에서 계속해 보겠습니다:

In [4]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors = 'coerce')
data['yyyymm']=data['InvoiceDate'].dt.strftime('%Y%m')
data['revenue'] = data['UnitPrice'] * data['Quantity']

revenue_df = data.groupby(['yyyymm'])['revenue'].sum().reset_index()
revenue_df.head()

,yyyymm,revenue
0,201012,748957.020
1,201101,560000.260
2,201102,498062.650
3,201103,683267.080
4,201104,493207.121


In [5]:
import plotly.express as px

In [6]:
revenue_df = revenue_df.sort_values(by='yyyymm')

In [7]:
fig = px.bar(revenue_df, x="yyyymm", y="revenue",
title="Monthly Revenue")
fig.update_xaxes(type='category')
fig.show()

월별 active cutomer 계산

In [8]:
active_customer_df = data.groupby(['yyyymm'])['CustomerID'].nunique().reset_index()
active_customer_df.columns = ['yyyymm','No of Active customers']
active_customer_df.head()

,yyyymm,No of Active customers
0,201012,948
1,201101,783
2,201102,798
3,201103,1020
4,201104,899


In [9]:
fig = px.bar(active_customer_df, x="yyyymm", y="No of Active customers",
title="Monthly Active customers")
fig.update_xaxes(type='category')
fig.show()

Model

고객 생애 가치(CLTV)는 고객과 기업의 라이프사이클과 관련된 순이익으로 정의됩니다. 

간단히 말해, CLV/LTV는 각 고객의 비즈니스 가치에 대한 반영입니다.

생애 가치를 예측하는 방법에는 여러 가지가 있습니다. 

하나는 회귀 문제인 고객의 가치를 예측하는 방법이고, 다른 하나는 분류 문제인 고객 그룹을 예측하는 방법일 수 있습니다. 

이 연습에서는 후자의 접근 방식을 사용합니다.

For this exercise, we will segment customers into the following groups:  
• Low LTV: Less active or low revenue customers  
• Mid-LTV: Fairly active and moderate revenue customers  
• High LTV: High revenue customers – the segment that we don't want to lose

3개월 분량의 데이터를 사용하여 고객의 최근성(R), 빈도(F), 금액(M) 지표를 계산하여 기능을 생성할 것입니다.   

이러한 기능을 확보한 후에는 6개월 분량의 데이터를 사용하여 모든 고객의 수익을 계산하고 LTV 클러스터 라벨(낮은 LTV, 중간 LTV, 높은 LTV)을 생성할 것입니다.   

그런 다음 생성된 레이블과 기능을 사용하여 신규 고객 그룹을 예측하는 데 사용할 수 있는 XGBoost 모델을 훈련합니다.

Feature engineering

이제 같은 노트북에서 작업을 계속하여 고객에 대한 R, F 및 M 값을 계산하고 개별 R, F 및 M 점수에서 계산된 값을 기준으로 고객을 그룹화해 보겠습니다:  
- Recency(R): 최근성 지표는 고객이 마지막으로 구매한 후 경과한 일수를 나타냅니다.  
- Frequency(F): 용어에서 알 수 있듯이 F는 고객이 구매를 한 횟수를 나타냅니다.  
- MOnetary(M): 특정 고객이 가져온 수익입니다.  

고객의 지출 및 구매 패턴은 인구 통계학적 위치에 따라 다르므로 이 연습에서는 영국에 속하는 데이터만 고려하겠습니다.  OnlineRetail.csv 파일을 읽고 다음과 같은 데이터를 필터링해 보겠습니다. 데이터를 필터링해 보겠습니다:

In [10]:
data = pd.read_csv("data/OnlineRetail.csv", encoding="unicode_escape")
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors = 'coerce')

In [11]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [12]:
uk_data = data.query("Country=='United Kingdom'").reset_index(drop=True)

In [13]:
uk_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


다음 코드 블록에서는 서로 다른 두 개의 데이터 프레임을 생성합니다.  
첫 번째 데이터 프레임(uk_data_3m)은 2011-03-01에서 2011-06-01 사이의 InvoiceDate에 대한 것입니다.   

이 데이터프레임은 RFM 기능을 생성하는 데 사용됩니다.  
두 번째 데이터 프레임(uk_data_6m)은 2011-06-01에서 2011-12-01 사이의 InvoiceDate에 대한 데이터 프레임입니다.

이 데이터프레임은 모델 학습을 위한 타겟 열을 생성하는 데 사용됩니다.  
이 연습에서 타겟 열은 LTV 그룹/클러스터입니다.  
고객 LTV 그룹을 계산하기 때문에 시간 간격이 클수록 더 나은 그룹을 만들 수 있습니다.  
따라서 6개월 분량의 데이터를 사용하여 LTV 그룹 레이블을 생성하겠습니다:

In [14]:
from datetime import datetime, timedelta, date

In [15]:
t1 = pd.Timestamp("2011-06-01 00:00:00.054000")
t2 = pd.Timestamp("2011-03-01 00:00:00.054000")
t3 = pd.Timestamp("2011-12-01 00:00:00.054000")
uk_data_3m = uk_data[(uk_data.InvoiceDate < t1) & (uk_data.InvoiceDate >= t2)].reset_index(drop=True)
uk_data_6m = uk_data[(uk_data.InvoiceDate >= t1) & (uk_data.InvoiceDate < t3)].reset_index(drop=True)

In [16]:
uk_data_3m

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,545220,21955,DOORMAT UNION JACK GUNS AND ROSES,2,2011-03-01 08:30:00,7.95,14620.0,United Kingdom
1,545220,48194,DOORMAT HEARTS,2,2011-03-01 08:30:00,7.95,14620.0,United Kingdom
2,545220,22556,PLASTERS IN TIN CIRCUS PARADE,12,2011-03-01 08:30:00,1.65,14620.0,United Kingdom
3,545220,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,2011-03-01 08:30:00,4.95,14620.0,United Kingdom
4,545220,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,2011-03-01 08:30:00,3.75,14620.0,United Kingdom
...,...,...,...,...,...,...,...,...
95188,555150,22895,SET OF 2 TEA TOWELS APPLE AND PEARS,3,2011-05-31 15:53:00,3.25,NaN,United Kingdom
95189,555150,22983,CARD BILLBOARD FONT,3,2011-05-31 15:53:00,0.42,NaN,United Kingdom
95190,555150,23169,CLASSIC GLASS COOKIE JAR,3,2011-05-31 15:53:00,4.15,NaN,United Kingdom
95191,555150,23298,SPOTTY BUNTING,5,2011-05-31 15:53:00,4.95,NaN,United Kingdom


이제 두 개의 서로 다른 데이터프레임이 생겼으므로 uk_data_3m 데이터프레임을 사용하여 RFM 값을 계산해 보겠습니다.   
다음 코드 블록은 UnitPrice에 Quantity를 곱하여 수익 열을 계산합니다.  
RFM 값을 계산하기 위해 코드 블록은 CustomerID에 대해 세 가지 집계를 수행합니다:

- R을 계산하려면 데이터 프레임의 max_date를 계산해야 하며, 모든 고객에 대해 R = max_date - x.max()를 계산해야 합니다. 여기서 x.max()는 특정 CustomerID의 최신 InvoiceDate를 계산합니다.

- F를 계산하려면 특정 CustomerID에 대한 인보이스 수를 계산합니다.

- M을 계산하려면 특정 CustomerID에 대한 매출의 합계 값을 구합니다.

In [17]:
uk_data_3m['revenue'] = uk_data_3m['UnitPrice'] * uk_data_3m['Quantity']
max_date = uk_data_3m['InvoiceDate'].max() + timedelta(days=1)
rfm_data = uk_data_3m.groupby(['CustomerID']).agg({
        'InvoiceDate': lambda x: (max_date - x.max()).days,
        'InvoiceNo': 'count',
        'revenue': 'sum'})

In [18]:
rfm_data.head()

,InvoiceDate,InvoiceNo,revenue
CustomerID,,,
12747.0,7,35,1082.09
12748.0,1,582,4336.73
12749.0,8,54,782.10
12821.0,23,6,92.72
12823.0,63,1,459.00


In [19]:
rfm_data.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'revenue': 'MonetaryValue'}, inplace=True)

In [20]:
rfm_data.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747.0,7,35,1082.09
12748.0,1,582,4336.73
12749.0,8,54,782.10
12821.0,23,6,92.72
12823.0,63,1,459.00


여기서는 고객에 대한 R, F, M 값을 계산했습니다. 다음으로 고객을 R, F, M 그룹으로 나누어야 합니다. 

이 그룹화는 R, F, M 지표의 관점에서 다른 고객에 대한 고객의 위치를 정의합니다. R, F, M 그룹을 계산하기 위해 고객을 각각 R, F, M 값에 따라 동일한 크기의 그룹으로 나눕니다. 이러한 값은 이전 코드 블록에서 계산되었습니다.

이를 위해 데이터 프레임에서 pd.qcut(https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html)이라는 메서드를 사용합니다. 

또는 클러스터링 방법을 사용하여 고객을 여러 그룹으로 나눌 수 있습니다. R, F, M 그룹의 값을 합산하여 0에서 9까지의 범위인 RFMScore라는 단일 값을 생성합니다.

In [21]:
r_grp = pd.qcut(rfm_data['Recency'], q=4, labels=range(3,-1,-1))
f_grp = pd.qcut(rfm_data['Frequency'], q=4, labels=range(0,4))
m_grp = pd.qcut(rfm_data['MonetaryValue'], q=4, labels=range(0,4))
rfm_data = rfm_data.assign(R=r_grp.values).assign(F=f_grp.values).assign(M=m_grp.values)
rfm_data['R'] = rfm_data['R'].astype(int)
rfm_data['F'] = rfm_data['F'].astype(int)
rfm_data['M'] = rfm_data['M'].astype(int)

In [22]:
rfm_data.head()

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12747.0,7,35,1082.09,3,2,3
12748.0,1,582,4336.73,3,3,3
12749.0,8,54,782.10,3,3,3
12821.0,23,6,92.72,2,0,0
12823.0,63,1,459.00,0,0,2


In [23]:
rfm_data['RFMScore'] = rfm_data['R'] + rfm_data['F'] + rfm_data['M']

In [24]:
rfm_data.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFMScore
CustomerID,,,,,,,
12747.0,7,35,1082.09,3,2,3,8
12748.0,1,582,4336.73,3,3,3,9
12749.0,8,54,782.10,3,3,3,9
12821.0,23,6,92.72,2,0,0,2
12823.0,63,1,459.00,0,0,2,2


In [25]:
rfm_data.groupby('RFMScore')[['Recency','Frequency','MonetaryValue']].mean()

,Recency,Frequency,MonetaryValue
RFMScore,,,
0,76.633588,5.206107,94.023359
1,58.500000,9.074074,158.990864
2,54.380208,13.968750,239.346354
3,38.091633,16.605578,271.437331
4,36.647619,24.180952,363.867000
5,34.000000,33.558252,535.423204
6,28.516432,44.920188,742.487324
7,18.431138,51.317365,974.310120
8,14.316547,80.064748,2192.578568


이 연습에서는 고객을 네 그룹으로 나눕니다.  
엘보우 방법을 사용하여 모든 데이터 집합에 대한 최적의 그룹 수를 계산할 수 있습니다. 

앞의 링크에는 최적의 그룹 수를 계산하는 데 사용할 수 있는 대체 방법에 대한 정보도 포함되어 있으므로 자유롭게 사용해 보세요.

In [26]:
rfm_data['Segment'] = 'Low-Value'
rfm_data.loc[rfm_data['RFMScore']>4,'Segment'] = 'Mid-Value' 
rfm_data.loc[rfm_data['RFMScore']>6,'Segment'] = 'High-Value' 
rfm_data = rfm_data.reset_index()

In [27]:
rfm_data.head()

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Segment
0,12747.0,7,35,1082.09,3,2,3,8,High-Value
1,12748.0,1,582,4336.73,3,3,3,9,High-Value
2,12749.0,8,54,782.10,3,3,3,9,High-Value
3,12821.0,23,6,92.72,2,0,0,2,Low-Value
4,12823.0,63,1,459.00,0,0,2,2,Low-Value


### Customer LTV

이제 3개월 분량의 데이터가 포함된 데이터 프레임에 고객에 대한 RFM 기능이 준비되었으므로, 이전과 마찬가지로 6개월 분량의 데이터(uk_data_6m)를 사용하여 고객의 매출을 계산하고 새로 만든 매출 데이터 프레임에 RFM 기능을 병합해 보겠습니다:

In [28]:
# Calculate revenue using the six month dataframe.
uk_data_6m['revenue'] = uk_data_6m['UnitPrice'] * uk_data_6m['Quantity']
revenue_6m = uk_data_6m.groupby(['CustomerID']).agg({
        'revenue': 'sum'})
revenue_6m.rename(columns={'revenue': 'Revenue_6m'}, 
                  inplace=True)
revenue_6m = revenue_6m.reset_index()
revenue_6m = revenue_6m.dropna()

In [29]:
# Merge the 6m revenue data frame with RFM data.
merged_data = pd.merge(rfm_data, revenue_6m, how="left")
merged_data.fillna(0)

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Segment,Revenue_6m
0,12747.0,7,35,1082.09,3,2,3,8,High-Value,1666.11
1,12748.0,1,582,4336.73,3,3,3,9,High-Value,18679.01
2,12749.0,8,54,782.10,3,3,3,9,High-Value,2323.04
3,12821.0,23,6,92.72,2,0,0,2,Low-Value,0.00
4,12823.0,63,1,459.00,0,0,2,2,Low-Value,765.00
...,...,...,...,...,...,...,...,...,...,...
1835,18272.0,21,59,966.74,2,3,3,8,High-Value,1730.16
1836,18273.0,66,1,51.00,0,0,0,0,Low-Value,102.00
1837,18280.0,86,10,180.60,0,0,0,0,Low-Value,0.00
1838,18283.0,9,100,217.15,3,3,1,7,High-Value,1351.83


수익_6m을 RFMScore에 대해 자유롭게 플롯합니다. 둘 사이에 양의 상관관계가 있음을 알 수 있습니다.

플로우 코드 블록에서 고객의 생애 가치인 revenue_6m 열을 사용하고 있습니다. 열을 사용하고, 낮은 LTV, 중간 LTV, 높은 LTV라는 세 개의 그룹을 만듭니다. 라는 세 개의 그룹을 생성합니다. 다시 한 번, 앞서 설명한 방법을 사용하여 최적의 클러스터 수를 확인할 수 있습니다:

In [30]:
uk_data_6m['revenue'] = uk_data_6m['UnitPrice'] * uk_data_6m['Quantity']
revenue_6m = uk_data_6m.groupby(['CustomerID']).agg({
        'revenue': 'sum'})
revenue_6m.rename(columns={'revenue': 'Revenue_6m'}, inplace=True)

In [31]:
revenue_6m = revenue_6m.reset_index()

In [32]:
merged_data = pd.merge(rfm_data, revenue_6m, how="left")
merged_data.fillna(0)

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Segment,Revenue_6m
0,12747.0,7,35,1082.09,3,2,3,8,High-Value,1666.11
1,12748.0,1,582,4336.73,3,3,3,9,High-Value,18679.01
2,12749.0,8,54,782.10,3,3,3,9,High-Value,2323.04
3,12821.0,23,6,92.72,2,0,0,2,Low-Value,0.00
4,12823.0,63,1,459.00,0,0,2,2,Low-Value,765.00
...,...,...,...,...,...,...,...,...,...,...
1835,18272.0,21,59,966.74,2,3,3,8,High-Value,1730.16
1836,18273.0,66,1,51.00,0,0,0,0,Low-Value,102.00
1837,18280.0,86,10,180.60,0,0,0,0,Low-Value,0.00
1838,18283.0,9,100,217.15,3,3,1,7,High-Value,1351.83


수익_6m을 RFMScore에 대해 자유롭게 플롯합니다. 둘 사이에 양의 상관관계가 있음을 알 수 있습니다.

플로우 코드 블록에서 고객의 생애 가치인 revenue_6m 열을 사용하고 있습니다. 열을 사용하고, 낮은 LTV, 중간 LTV, 높은 LTV라는 세 개의 그룹을 만듭니다. 라는 세 개의 그룹을 생성합니다. 다시 한 번, 앞서 설명한 방법을 사용하여 최적의 클러스터 수를 확인할 수 있습니다:

In [33]:
from sklearn.cluster import KMeans

In [34]:
# Create LTV cluster groups
merged_data = merged_data[merged_data['Revenue_6m']<merged_data['Revenue_6m'].quantile(0.99)]
kmeans = KMeans(n_clusters=3)
kmeans.fit(merged_data[['Revenue_6m']])
merged_data['LTVCluster'] = kmeans.predict(merged_data[['Revenue_6m']])
merged_data.groupby('LTVCluster')['Revenue_6m'].describe()

/Users/sguys99/anaconda3/envs/feast_book/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



,count,mean,std,min,25%,50%,75%,max
LTVCluster,,,,,,,,
0,1170.0,828.670189,621.405282,-609.40,324.4675,663.525,1245.465,2447.57
1,21.0,14123.309048,3653.311262,9313.18,11464.6700,12913.990,16756.310,20530.00
2,186.0,4137.019892,1477.396388,2503.30,2964.2275,3635.200,4987.535,8910.04


In [35]:
merged_data.head()

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Segment,Revenue_6m,LTVCluster
0,12747.0,7,35,1082.09,3,2,3,8,High-Value,1666.11,0
1,12748.0,1,582,4336.73,3,3,3,9,High-Value,18679.01,1
2,12749.0,8,54,782.10,3,3,3,9,High-Value,2323.04,0
4,12823.0,63,1,459.00,0,0,2,2,Low-Value,765.00,0
7,12836.0,28,62,814.71,1,3,3,7,High-Value,951.46,0


보시다시피 레이블 1이 있는 클러스터에는 그룹의 평균 수익이 $14,123.309이므로 평생 가치가 매우 높은 고객 그룹이 포함되어 있지만 이러한 고객은 21명에 불과합니다. 

레이블이 0인 클러스터에는 그룹의 평균 수익이 $828.67에 불과하여 평생 가치가 낮은 고객 그룹이 포함되어 있지만 이러한 고객은 1,170명에 불과합니다. 

이 그룹화를 통해 어떤 고객을 항상 만족시켜야 하는지에 대한 아이디어를 얻을 수 있습니다.

### The feature set and model

In [36]:
feature_data = pd.get_dummies(merged_data)

In [37]:
feature_data.head()

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Revenue_6m,LTVCluster,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value
0,12747.0,7,35,1082.09,3,2,3,8,1666.11,0,True,False,False
1,12748.0,1,582,4336.73,3,3,3,9,18679.01,1,True,False,False
2,12749.0,8,54,782.10,3,3,3,9,2323.04,0,True,False,False
4,12823.0,63,1,459.00,0,0,2,2,765.00,0,False,True,False
7,12836.0,28,62,814.71,1,3,3,7,951.46,0,True,False,False


이제 이 기능 세트를 사용하여 Xgboost 모델을 훈련해 보겠습니다. 예측 레이블(y)은 LTVCluster 열이고, Revenue_6m 및 CustomerID 열을 제외한 나머지 데이터 집합은 X 값입니다. 

LTVCluster 열(y)이 Revenue_6m을 사용하여 계산되므로 Revenue_6m은 기능 집합에서 삭제됩니다. 신규 고객의 경우 최소 6개월 분량의 데이터 없이도 다른 기능을 계산할 수 있으며 없이도 다른 기능을 계산할 수 있으며 LTVCluster(y)도 예측할 수 있습니다. 

다음 코드는 Xgboost 모델을 학습시킵니다:

In [38]:
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [39]:
#Splitting data into train and test data set.
X = feature_data.drop(['CustomerID', 'LTVCluster',
                       'Revenue_6m'], axis=1)
y = feature_data['LTVCluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
xgb_classifier = xgb.XGBClassifier(max_depth=5, objective='multi:softprob')

In [40]:
xgb_model = xgb_classifier.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

In [41]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2,
       2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0])

In [42]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.90      0.88       115
           1       0.00      0.00      0.00         2
           2       0.28      0.24      0.26        21

    accuracy                           0.78       138
   macro avg       0.38      0.38      0.38       138
weighted avg       0.77      0.78      0.78       138

